In [1]:

import numpy as np
import openhd as hd

/home/hpc/iwi3/iwi3083h/.local/lib/python3.8/site-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [2]:
!rm -rf ~/.openhd

In [3]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = False,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

In [4]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, 
                         batch_size=10000, 
                         shuffle=True, 
                         num_workers=1)

In [5]:
test_loader = DataLoader(test_data, 
                         batch_size=50,
                         shuffle=True, 
                         num_workers=1)

In [6]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output
    
    def extract_conv_features(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x


In [7]:
cnn = CNN()


In [8]:




loss_func = nn.CrossEntropyLoss()   


from torch import optim
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)   

from torch.autograd import Variable

def train(num_epochs, cnn):
    
    cnn.train()
    total_step = len(train_loader)
        
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y

            output = cnn(b_x)             
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

num_epochs = 2
train(num_epochs, cnn)


def test():
    # Test the model
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            test_output = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            pass
        
        print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    pass
test()


Test Accuracy of the model on the 10000 test images: 0.88


In [9]:
images, labels = next(iter(train_loader))

# feature_matrix = feature_matrix.reshape((60000, 28*28))

# feature_matrix = np.array(feature_matrix, dtype = np.float32)
# labels = np.array(labels, dtype=np.int32)

feature_matrix = cnn.extract_conv_features(images).detach().numpy()

feature_matrix = feature_matrix.reshape((10000, 1568))

feature_matrix = np.array(feature_matrix, dtype = np.float32)
labels = np.array(labels, dtype=np.int32)

In [10]:
n_classes = len(set(labels))

In [11]:
images_tst, labels_tst = next(iter(test_loader))

In [12]:
images_tst.shape

torch.Size([50, 1, 28, 28])

In [25]:
%%timeit
feature_matrix_tst = cnn.extract_conv_features(images_tst).detach().numpy()

feature_matrix_tst = feature_matrix_tst.reshape((50, 1568))

feature_matrix_tst = np.array(feature_matrix_tst, dtype = np.float32)
# labels_tst = np.array(labels_tst, dtype=np.int32)

1.82 ms ± 3.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
1.82 / 1000

0.00182

In [14]:
Q = 10
D = 10000
hd.init(D=D, context=globals())

In [15]:
N = feature_matrix.shape[0]
F = feature_matrix.shape[1]
n_classes = 10
N, F

(10000, 1568)

In [16]:
@hd.run
def create_random_bases():
    id_base = hd.draw_random_hypervector()
    level_base = hd.draw_random_hypervector()
    return id_base, level_base


@hd.run
def create_ids(F, id_base):
    id_hvs = hd.hypermatrix(F) # np.zeros(F, N) (not the empty list) 
    for f in range(F):
        id_hvs[f] = hd.permute(id_base, f)

    return id_hvs

@hd.run
def create_levels(Q, level_base):
    level_hvs = hd.hypermatrix(Q+1) # np.zeros((Q+1), N) (not the empty list)
    for q in range(Q+1):
        idx = int(q/float(Q) * D) / 2
        level_hvs[q] = hd.flip(level_base, idx)
        level_hvs[q] = hd.shuffle(level_hvs[q], 0)

    return level_hvs

In [17]:
with hd.utils.timing("Base hypervectors"):
    id_base, level_base = create_random_bases()
    id_hvs = create_ids(F, id_base)
    level_hvs = create_levels(Q, level_base)


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [__ARG__id_base : __one__]
  <class '_ast.Assign'> :  [__ARG__level_base : __one__]

{'id_base': 'hypervec_type', 'level_base': 'hypervec_type', '__ARG__level_base': 'float*', '__ARG__id_base': 'float*', 'id_base_22b19319': <class 'float'>, 'level_base_5541502b': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [__ARG__id_hvs : __ARG__id_base]

{'f': <class 'int'>, 'F': <class 'int'>, 'id_hvs': 'hypermat_type', 'id_base': 'hypervec_type', '__ARG__id_base': 'float*', '__ARG__id_hvs': 'float*', 'id_base_f_16a56737': <class 'float'>, 'id_hvs_f_00177a23': <class 'float'>}
[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [level_hvs_q_7acb546c : __ARG__level_base]
  <class '_ast.Assign'> :  [__ARG__level_hvs : level_hvs_q_7acb546c]

{'idx': <class 'int'>, 'q': 'int', 'Q': <class 'int'>, 'D': 

In [18]:
def preprocesser(
        org_feature, cnv_feature, # Predefined argument (single feature)
        Q, level_hvs, id_hvs): # arguments passed by args
    cnv_feature = int(org_feature * Q)


def encoder(
        input_features, output_hypervector, # Predefined arguments
        Q, level_hvs, id_hvs): # arguments passed by args
    for f in range(F):
        output_hypervector += level_hvs[input_features[f]] * id_hvs[f]

In [19]:
with hd.utils.timing("Encode training"):
    hv_matrix = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = feature_matrix,
#             preprocess_function = preprocesser # optional
            )

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'original_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'preprocessed_feature': <class 'float'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int'>, 

In [20]:
feature_matrix_tst.shape

(50, 1568)

In [21]:
with hd.utils.timing("Encode testing"):
    hv_matrix_tst = hd.encode(
            encoder, extra_args = (Q, level_hvs, id_hvs),
            feature_matrix = feature_matrix_tst,
            preprocess_function = preprocesser # optional
            )


[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__output_hypervector]
  <class '_ast.For'> :  <RPT: F>
    <class '_ast.Assign'> :  [output_hypervector___n_____base_n___2964e176 : __ARG__id_hvs*__ARG__level_hvs + output_hypervector___n_____base_n___2964e176]
  <class '_ast.Assign'> :  [__ARG__output_hypervector : output_hypervector___n_____base_n___2964e176]

{'__n__': <class 'int'>, '__blockIdx_y__': <class 'int'>, '__base_n__': 'int', '__N__': <class 'int'>, '__blockDim_x__': <class 'int'>, '__F__': <class 'int'>, '__threadIdx_x__': <class 'int'>, 'F_PER_THREAD': <class 'int'>, 'sample_idx_in_stream': <class 'int'>, '__stream__': 'int', '__M__': <class 'int'>, '__f__': <class 'int'>, '__f_idx__': <class 'int'>, 'org_feature': <class 'float'>, 'input_features': 'np_float_array_type', 'cnv_feature': <class 'int'>, 'Q': <class 'int'>, '__shared_features__': 'np_float_array_type', '__d__': <class 'int

In [28]:
@hd.run
def single_pass(hv_matrix, labels, N, n_classes):
    class_hvs = hd.hypermatrix(n_classes)

    for idx in range(N):
        class_hvs[labels[idx]] += hv_matrix[idx]

    return class_hvs

In [29]:
with hd.utils.timing("Single pass"):
    class_hvs = single_pass(hv_matrix, labels, N, n_classes)
    class_hvs.debug_print_values()

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + __ARG__hv_matrix]

{'idx': <class 'int'>, 'N': <class 'int'>, 'class_hvs': 'hypermat_type', 'labels': 'np_int_array_type', 'hv_matrix': 'hypermat_type', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', '__ARG__class_hvs': 'float*', '__ARG__hv_matrix': 'float*', 'hv_matrix_idx_ead026f6': <class 'float'>, 'labels_idx_b9c77ce4': <class 'int'>, 'class_hvs_labels_idx_df62a0b1': <class 'float'>}
Single pass	0.5503640174865723


In [30]:
def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


In [31]:
@hd.run
def retrain(class_hvs, hv_matrix, labels, N, n_classes):
    search_results = hd.search(class_hvs, hv_matrix)

    for idx in range(N):
        if search_results[idx] != labels[idx]:
            class_hvs[labels[idx]] += hv_matrix[idx]
            class_hvs[search_results[idx]] -= hv_matrix[idx]

    return class_hvs


In [32]:
RETRAIN_ITERATIONS = 100
SHOW_STEP_RESULT = False
for it in range(RETRAIN_ITERATIONS):
    with hd.utils.timing("Retrain itereation: %d" % it):
        class_hvs = retrain(class_hvs, hv_matrix, labels, N, n_classes)

    if SHOW_STEP_RESULT and labels_tst is not None:
        validate(labels_tst, hd.search(class_hvs, hv_matrix_tst).to_numpy())

[ERROR]	jit.date_type_mutator	
<class '_ast.Module'> : 
  <class '_ast.For'> :  <RPT: N>
    <class '_ast.Assign'> :  [hv_matrix_idx_ead026f6 : __ARG__hv_matrix]
    <class '_ast.If'> : 
      then : 
        <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + hv_matrix_idx_ead026f6]
        <class '_ast.Assign'> :  [__ARG__class_hvs : __ARG__class_hvs + hv_matrix_idx_ead026f6]
      else : 

{'idx': <class 'int'>, 'N': <class 'int'>, 'search_results': 'np_int_array_type', 'labels': 'np_int_array_type', 'class_hvs': 'hypermat_type', 'hv_matrix': 'hypermat_type', '__ARG__labels': 'int*', '__ARG__labels__STRIDE__': 'const int', '__ARG__search_results': 'int*', '__ARG__search_results__STRIDE__': 'const int', '__ARG__class_hvs': 'float*', '__ARG__hv_matrix': 'float*', 'search_results_idx_b85d35f6': <class 'int'>, 'labels_idx_b9c77ce4': <class 'int'>, 'hv_matrix_idx_ead026f6': <class 'float'>, 'class_hvs_labels_idx_df62a0b1': <class 'float'>, 'class_hvs_idx_search_results_572d84

In [33]:
@hd.run
def assoc_search(class_hvs, hv_matrix_tst):
    ret = hd.search(class_hvs, hv_matrix_tst)
    return ret

In [34]:
with hd.utils.timing("Testing with class model\n"):
    search_results = assoc_search(class_hvs, hv_matrix)

Testing with class model
	0.020494461059570312


In [ ]:
search_results.to_numpy() == labels_tst

In [ ]:
def validate(labels, pred_labels):
    n_correct = (pred_labels == labels).sum()
    n_labels = len(labels)
    print(n_correct, n_labels, n_correct / float(n_labels) * 100)


In [ ]:
validate(labels, search_results.to_numpy())


In [ ]:
feature_matrix.shape

In [ ]:
labels

In [ ]:
search_results.to_numpy()